In [19]:
import numpy as np
import pandas as pd
import sklearn as sk
import sklearn.model_selection
import sklearn.neural_network

In [40]:
features = pd.read_csv('hb_train_feature.csv')
truths = pd.read_csv('hb_train_truth.csv')

In [4]:
# add features to data

In [42]:
# add unique NE feature
!python -m pip install nashpy
import nashpy
import warnings
warnings.filterwarnings('ignore')

def add_NE(features):
    UNEs, NE_cnts = [], []
    for i in range(features.shape[0]):
        payoffs = np.array(features.iloc[i]) 
        R = payoffs[:9].reshape(3,3)
        C = payoffs[9:18].reshape(3,3)
        rps = nashpy.Game(R, C)
        eqs = list(rps.support_enumeration()) # could be unique or multiple (PNE MNE)
        UNE = list(np.concatenate(eqs[0])) if len(eqs)==1 else list(np.zeros(6))
        NE_cnt = len(eqs)    
        UNEs.append(UNE)
        NE_cnts.append(NE_cnt)

    # append to features
    names = ['UNE_r1', 'UNE_r2','UNE_r3','UNE_c1','UNE_c2','UNE_c3']

    for i in range(6):
        features[names[i]] = [UNE[i] for UNE in UNEs]
    features['NE_cnts'] = NE_cnts
add_NE(features)
features.head()

,r11,r12,r13,r21,r22,r23,r31,r32,r33,c11,...,c31,c32,c33,UNE_r1,UNE_r2,UNE_r3,UNE_c1,UNE_c2,UNE_c3,NE_cnts
0,80.0,50.0,80.0,20.0,90.0,60.0,30.0,10.0,10.0,70.0,...,10.0,40.0,50.0,1.0,0.0,0.0,1.0,0.0,0.0,1
1,20.0,51.0,50.0,10.0,70.0,30.0,60.0,5.0,45.0,20.0,...,50.0,30.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,5
2,70.0,100.0,48.0,10.0,29.0,40.0,40.0,60.0,100.0,70.0,...,48.0,40.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,3
3,20.0,60.0,70.0,30.0,90.0,20.0,40.0,40.0,50.0,40.0,...,60.0,10.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,2
4,13.0,80.0,60.0,74.0,20.0,70.0,60.0,100.0,10.0,13.0,...,60.0,70.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,3


In [44]:
# add max max feature
def add_maxmax(features):
    features['max_max'] = features.iloc[:,:9].idxmax(axis=1).apply(lambda x: int(x[1]))
    features = pd.get_dummies(features, columns=['max_max'], drop_first=True)
add_maxmax(features)
features.head()

,r11,r12,r13,r21,r22,r23,r31,r32,r33,c11,...,c32,c33,UNE_r1,UNE_r2,UNE_r3,UNE_c1,UNE_c2,UNE_c3,NE_cnts,max_max
0,80.0,50.0,80.0,20.0,90.0,60.0,30.0,10.0,10.0,70.0,...,40.0,50.0,1.0,0.0,0.0,1.0,0.0,0.0,1,2
1,20.0,51.0,50.0,10.0,70.0,30.0,60.0,5.0,45.0,20.0,...,30.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,5,2
2,70.0,100.0,48.0,10.0,29.0,40.0,40.0,60.0,100.0,70.0,...,40.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,3,1
3,20.0,60.0,70.0,30.0,90.0,20.0,40.0,40.0,50.0,40.0,...,10.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2
4,13.0,80.0,60.0,74.0,20.0,70.0,60.0,100.0,10.0,13.0,...,70.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3


In [45]:
# # add minimax feature
def grid_form(row):
    return np.array(row).reshape(3, 3)

def is_argmax(arr, i):
    return arr[i] == max(arr)
               
def is_min_max(row, i):
    grid = grid_form(row)
    mins = np.min(grid, axis=0)
    if is_argmax(mins, i - 1):
        return 1
    return 0

def add_minimax(features):
    for i in range(1,4):
        features['min_max_{}'.format(i)] = 0
        for j in range(len(features)):
            features.loc[j,'min_max_{}'.format(i)] = is_min_max(features.iloc[j, :9], i)
add_minimax(features)
features.head()

,r11,r12,r13,r21,r22,r23,r31,r32,r33,c11,...,UNE_r2,UNE_r3,UNE_c1,UNE_c2,UNE_c3,NE_cnts,max_max,min_max_1,min_max_2,min_max_3
0,80.0,50.0,80.0,20.0,90.0,60.0,30.0,10.0,10.0,70.0,...,0.0,0.0,1.0,0.0,0.0,1,2,1,0,0
1,20.0,51.0,50.0,10.0,70.0,30.0,60.0,5.0,45.0,20.0,...,0.0,0.0,0.0,0.0,0.0,5,2,0,0,1
2,70.0,100.0,48.0,10.0,29.0,40.0,40.0,60.0,100.0,70.0,...,0.0,0.0,0.0,0.0,0.0,3,1,0,0,1
3,20.0,60.0,70.0,30.0,90.0,20.0,40.0,40.0,50.0,40.0,...,0.0,0.0,0.0,0.0,0.0,2,2,0,1,0
4,13.0,80.0,60.0,74.0,20.0,70.0,60.0,100.0,10.0,13.0,...,0.0,0.0,0.0,0.0,0.0,3,3,0,1,0


In [46]:
## add maximum payoff for both agents feature
def is_max_altruism(row, i):
    print(row)
    total_welfare = [row[i] + row[i + 9] for i in range(9)]
    if is_argmax(i):
        return 1
    return 0    

def add_maximin(features):
    for i in range(1,4):
        features['altruism_{}'.format(i)] = 0
        for j in range(len(features)):
            features.loc[j,'altruism_{}'.format(i)] = is_min_max(features.iloc[j, :9], i)
add_maximin(features)
features.head()

,r11,r12,r13,r21,r22,r23,r31,r32,r33,c11,...,UNE_c2,UNE_c3,NE_cnts,max_max,min_max_1,min_max_2,min_max_3,altruism_1,altruism_2,altruism_3
0,80.0,50.0,80.0,20.0,90.0,60.0,30.0,10.0,10.0,70.0,...,0.0,0.0,1,2,1,0,0,1,0,0
1,20.0,51.0,50.0,10.0,70.0,30.0,60.0,5.0,45.0,20.0,...,0.0,0.0,5,2,0,0,1,0,0,1
2,70.0,100.0,48.0,10.0,29.0,40.0,40.0,60.0,100.0,70.0,...,0.0,0.0,3,1,0,0,1,0,0,1
3,20.0,60.0,70.0,30.0,90.0,20.0,40.0,40.0,50.0,40.0,...,0.0,0.0,2,2,0,1,0,0,1,0
4,13.0,80.0,60.0,74.0,20.0,70.0,60.0,100.0,10.0,13.0,...,0.0,0.0,3,3,0,1,0,0,1,0


In [7]:
# splits into training and test sets
X_training, X_test = sk.model_selection.train_test_split(features)
y_training, y_test = truths[['f1', 'f2', 'f3']].loc[X_training.index], truths[['f1', 'f2', 'f3']].loc[X_test.index]

# standardizes features 
scaler = sk.preprocessing.StandardScaler()
scaler.fit_transform(X_training)
scaler.transform(X_test)
scaler.transform(X_training);
#X_training.head()

In [8]:
# trains separate neural net to predict each specific player's actions
classifier = sk.neural_network.MLPRegressor(solver='lbfgs', random_state=1, max_iter=1000000000, verbose=True)
classifier.fit(X_training, y_training)
classifier.score(X_test, y_test)


-6.640963506208292

In [28]:
# function to test # of top actions predicted correctly
top_actions_test = truths['action'].loc[y_test.index].reset_index(drop=True)
predicted_frequency = pd.DataFrame(classifier.predict(X_test))
predicted_actions = predicted_frequency.idxmax(axis=1).apply(lambda x: x + 1)

# number of correct top-action predictions on test data
result = (top_actions_test==predicted_actions).value_counts()
print("A =", result[1]/(result[0]+result[1]))

A = 0.6666666666666666


In [32]:
# cross_validation
from sklearn.model_selection import cross_val_score
mlp = sk.neural_network.MLPRegressor(solver='lbfgs', random_state=1, max_iter=1000000000, verbose=True)
scores = cross_val_score(classifier, features, truths)

print('Cross-Validation scores: {}'.format(scores))
print('Average score: {}'.format(np.mean(scores)))

Cross-Validation scores: [-3.33573427 -2.66680172 -5.95968093 -2.97258865 -3.83100883]
Average score: -3.753162878893167


In [33]:
# Apply model to the entire training data
mlp.fit(features, truths[['f1', 'f2', 'f3']])
pred_frequency = pd.DataFrame(mlp.predict(features))

# calculate A score on training data
truth_actions = truths['action']
pred_actions = pred_frequency.idxmax(axis=1).apply(lambda x: x + 1)
As = (truth_actions==pred_actions)
print("A =", np.mean(As))

# calculate Q score on training data
truth_freqency = truths[['f1', 'f2', 'f3']]
pred_frequency.columns = ['f1', 'f2', 'f3']
Qs = ((truth_freqency.subtract(pred_frequency, axis='column'))**2).sum(axis=1)
print("Q =", np.mean(Qs))

A = 0.928
Q = 0.011385296314785086


In [48]:
# create hb_test_pred
test_features = pd.read_csv('hb_test_feature.csv')
add_NE(test_features)
add_maxmax(test_features)
add_minimax(test_features)
add_maximin(test_features)
print(features.shape, test_features.shape)
output = pd.DataFrame(mlp.predict(test_features))
output['action'] = output.idxmax(axis=1).apply(lambda x: x + 1)
output.columns = ['f1', 'f2', 'f3', 'action']
#output
output.to_csv('hb_test_pred.csv', sep=',', encoding='utf-8', index=False)

(250, 32) (200, 32)
